# TSA Chapter 5: TGARCH Simulation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_tgarch_sim/TSA_ch5_tgarch_sim.ipynb)

TGARCH/GJR-GARCH simulation with asymmetric volatility response.

In [ ]:
!pip install numpy pandas matplotlib scipy -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Simulation helpers
def simulate_arch(n=500, omega=0.1, alpha=0.7, seed=42):
    np.random.seed(seed)
    sigma2 = np.zeros(n); eps = np.zeros(n)
    sigma2[0] = omega / max(1 - alpha, 0.01)
    for t in range(1, n):
        sigma2[t] = omega + alpha * eps[t-1]**2
        eps[t] = np.sqrt(max(sigma2[t], 1e-8)) * np.random.randn()
    return eps, np.sqrt(np.maximum(sigma2, 1e-8))

def simulate_garch(n=500, omega=0.05, alpha=0.1, beta=0.85, seed=42):
    np.random.seed(seed)
    sigma2 = np.zeros(n); eps = np.zeros(n)
    ab = alpha + beta
    sigma2[0] = omega / max(1 - ab, 0.01) if ab < 1 else omega
    for t in range(1, n):
        sigma2[t] = omega + alpha * eps[t-1]**2 + beta * sigma2[t-1]
        eps[t] = np.sqrt(max(sigma2[t], 1e-8)) * np.random.randn()
    return eps, np.sqrt(np.maximum(sigma2, 1e-8))

def simulate_egarch(n=500, omega=-0.1, alpha=0.2, gamma=-0.1, beta=0.95, seed=42):
    np.random.seed(seed)
    log_sigma2 = np.zeros(n); sigma = np.zeros(n); eps = np.zeros(n)
    E_abs_z = np.sqrt(2 / np.pi)
    log_sigma2[0] = omega / max(1 - beta, 0.01) if beta < 1 else omega
    sigma[0] = np.exp(log_sigma2[0] / 2)
    for t in range(1, n):
        z_prev = eps[t-1] / sigma[t-1] if sigma[t-1] > 0 else 0
        log_sigma2[t] = omega + alpha * (np.abs(z_prev) - E_abs_z) + gamma * z_prev + beta * log_sigma2[t-1]
        sigma[t] = np.exp(log_sigma2[t] / 2)
        eps[t] = sigma[t] * np.random.randn()
    return eps, sigma

def simulate_gjr_garch(n=500, omega=0.05, alpha=0.05, gamma=0.1, beta=0.85, seed=42):
    np.random.seed(seed)
    sigma2 = np.zeros(n); eps = np.zeros(n)
    denom = 1 - alpha - gamma / 2 - beta
    sigma2[0] = omega / max(denom, 0.01) if denom > 0 else omega
    for t in range(1, n):
        I = 1 if eps[t-1] < 0 else 0
        sigma2[t] = omega + alpha * eps[t-1]**2 + gamma * eps[t-1]**2 * I + beta * sigma2[t-1]
        eps[t] = np.sqrt(max(sigma2[t], 1e-8)) * np.random.randn()
    return eps, np.sqrt(np.maximum(sigma2, 1e-8))

In [ ]:
# TGARCH simulation
n = 300
eps_gjr, sigma_gjr = simulate_gjr_garch(n=n, alpha=0.03, gamma=0.12, beta=0.85)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(eps_gjr, color=BLUE, alpha=0.7, lw=0.8, label='Returns ($\\varepsilon_t$)')
ax.fill_between(range(n), -2*sigma_gjr, 2*sigma_gjr,
                color=RED, alpha=0.2, label='$\\pm 2\\sigma_t$ (conditional)')
ax.axhline(0, color=GRAY, ls='-', alpha=0.3)
ax.set_title('TGARCH/GJR-GARCH: Asymmetric Volatility Response\n'
             '$\\alpha^+ = 0.03$, $\\alpha^- = 0.15$', fontweight='bold')
ax.set_xlabel('t'); ax.set_ylabel('$\\varepsilon_t$')
ax.set_xlim(0, n)
legend_bottom(ax, ncol=2, y=-0.18)
fig.tight_layout(); fig.subplots_adjust(bottom=0.20)
save_chart(fig, 'garch_tgarch_simulation'); plt.show()